In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string
import datetime

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [5]:
sql_sla_season = 'select * from sladb.dbo.tbl_sla_season'

In [6]:
sql_season_definition = 'select * from sladb.dbo.tbl_ref_sla_season_definition'

In [7]:
sql_season_date = 'select * from sladb.dbo.tbl_sla_season_date'

In [8]:
sql_month_tbl = 'select * from tbl_ref_sla_season_month_name'

In [9]:
sql_unit_sla_season = 'select * from sladb.dbo.tbl_unit_sla_season'

## Execute all of the sql queries and store the results

In [10]:
month_tbl  = pd.read_sql(con = engine, sql = sql_month_tbl)

In [11]:
before_sla_season = pd.read_sql(con = engine, sql = sql_sla_season)

In [12]:
date_cols_season = [
    'effective_start', 'effective_end', 'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [13]:
# pd.to_datetime(before_sla_season['effective_start'])

In [14]:
for i in date_cols_season:
    before_sla_season[i] = pd.to_datetime(before_sla_season[i])

In [15]:
before_sla_season.head()

,season_id,season_desc,year_round,effective,effective_end,effective_end_adj,effective_start,effective_start_adj,created_date_utc,updated_date_utc
0,1,"Year-round, not seasonal",True,True,NaT,NaT,2014-01-01,2014-01-05,2020-09-10 21:01:16.263,2020-09-10 21:01:16.327
1,2,"Beaches, etc.",False,True,NaT,NaT,2019-07-01,2019-07-07,2020-09-10 21:01:16.467,2020-09-10 21:01:16.467
2,3,"Ballfields, etc.",False,True,NaT,NaT,2019-07-01,2019-07-07,2020-09-10 21:01:16.593,2020-09-10 21:01:16.593


In [16]:
before_season_definition = pd.read_sql(con = engine, sql = sql_season_definition)

In [17]:
before_season_definition.head()

,season_date_ref_id,season_id,date_ref_fixed,month_name_desc,date_ref_day_number,day_name_desc,day_rank_id,date_type_id
0,1,1,True,January,1,None,None,1
1,2,1,True,December,31,None,None,2
2,3,2,True,May,1,None,None,1
3,4,2,True,October,1,None,None,2
4,5,3,True,March,1,None,None,1


In [18]:
before_season_date = pd.read_sql(con = engine, sql = sql_season_date)

In [19]:
date_cols_season_date = [
    'effective_start', 'effective_end', 'effective_start_adj',
    'effective_end_adj'
]

In [20]:
for i in date_cols_season_date:
    before_season_date[i] = pd.to_datetime(before_season_date[i])

In [21]:
before_season_date.head()

,season_date_id,season_id,effective_start,effective_start_adj,effective_end,effective_end_adj,date_category_id
0,1,1,2014-01-01,2014-01-05,2021-09-10,2021-09-11,1
1,2,2,2019-07-01,2019-07-07,2019-10-01,2019-10-05,1
2,3,2,2019-10-02,2019-10-06,2020-04-30,2020-05-02,2
3,4,2,2020-05-01,2020-05-03,2020-10-01,2020-10-03,1
4,5,2,2020-10-02,2020-10-04,2021-04-30,2021-05-01,2


In [22]:
before_unit_season = pd.read_sql(con = engine, sql = sql_unit_sla_season)
before_unit_season.head()

,sla_season_id,unit_id,sla_code,season_id,effective,effective_start,effective_start_adj,effective_end,effective_end_adj,change_request_id,created_date_utc,updated_date_utc
0,1,RZ245,11,1,False,2014-01-01,2014-01-05,2020-03-21,2020-03-21,1,2020-09-10 21:01:19.123,2020-09-10 21:01:22.547
1,2,RZ25,11,1,False,2014-01-01,2014-01-05,2020-03-21,2020-03-21,2,2020-09-10 21:01:19.123,2020-09-10 21:01:22.547
2,3,RZ255,11,1,False,2014-01-01,2014-01-05,2020-03-21,2020-03-21,3,2020-09-10 21:01:19.123,2020-09-10 21:01:22.547
3,4,RZ258,11,1,False,2014-01-01,2014-01-05,2020-03-21,2020-03-21,4,2020-09-10 21:01:19.123,2020-09-10 21:01:22.547
4,5,RZ26,11,1,False,2014-01-01,2014-01-05,2020-03-21,2020-03-21,5,2020-09-10 21:01:19.123,2020-09-10 21:01:22.547


In [23]:
date_cols_unit_season = [
    'effective_start', 'effective_end', 'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [24]:
for i in date_cols_unit_season:
    before_unit_season[i] = pd.to_datetime(before_unit_season[i])

In [25]:
timing = []

In [26]:
descriptions = []

In [27]:
results = []

## Before Tests

### First before test

In [28]:
timing.append('before')

In [29]:
descriptions.append(
    '''Ensure that 'effective'==1 for every season that has 'effective_end_date' == null'''
)

In [30]:
results.append(
    len(before_sla_season[(pd.isnull(before_sla_season['effective_end']))
                          & (before_sla_season['effective'] == False)]) == 0)

### Second before test

In [31]:
timing.append('before')

In [32]:
descriptions.append(
    '''Test that 'effective_end_adj' is after today if 'effective'==1 AND \
'effective_end' != null in tbl_sla_season''')

In [33]:
before_sla_season['effective_end_adj'] = pd.to_datetime(
    before_sla_season['effective_end_adj'])

In [34]:
today = datetime.datetime.today()

In [35]:
temp_df = before_sla_season[(before_sla_season['effective'] == True)
                  & (~pd.isnull(before_sla_season['effective_end']))].copy()

In [36]:
results.append(len(temp_df[temp_df['effective_end_adj'] > today])==len(temp_df))

### Third before test

In [37]:
timing.append('before')

In [38]:
descriptions.append('''Test that every season in tbl_sla_season has \
exactly two entries in tbl_ref_sla_season_definition''')

In [39]:
entry_count = []
for i in before_sla_season.season_id.values:
    entry_count.append(
        len(before_season_definition[before_season_definition['season_id'] ==
                                     i]) == 2)

In [40]:
results.append(False not in entry_count)

### Fourth before test

In [41]:
timing.append('before')

In [42]:
descriptions.append('''Test that in tbl_ref_sla_season_definition if \
'date_ref_fixed'==True then columns 'month_name_desc' and 'date_ref_day_number' \
are not null''')

In [43]:
temp_df = before_season_definition[before_season_definition['date_ref_fixed']
                                   == True]

In [44]:
results.append(
    len(temp_df) == len(temp_df[~(pd.isnull(temp_df['month_name_desc']))
                                &
                                (~pd.isnull(temp_df['date_ref_day_number']))]))

### Fifth before test

In [45]:
timing.append('before')

In [46]:
descriptions.append('''Test that in tbl_ref_sla_season_definition if \
'date_ref_fixed'==False then columns 'month_name_desc' and 'day_name_desc' \
and 'day_rank_id' are not null''')

In [47]:
temp_df = before_season_definition[before_season_definition['date_ref_fixed']==False]

In [48]:
results.append(
    len(temp_df) == len(temp_df[~(pd.isnull(temp_df['month_name_desc']))
                                & (~pd.isnull(temp_df['day_name_desc']))
                                & (~pd.isnull(temp_df['day_rank_id']))]))

### Sixth before test

In [49]:
timing.append('before')

In [50]:
descriptions.append('''For year-round seasons, test that 'date_ref_fixed'==1, \
and start/end dates are Jan 1/Dec 31''')

In [51]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == True],
    on='season_id')[[
        'season_id', 'year_round', 'date_ref_fixed', 'month_name_desc',
        'date_ref_day_number', 'date_type_id'
    ]]
temp_df

,season_id,year_round,date_ref_fixed,month_name_desc,date_ref_day_number,date_type_id
0,1,True,True,January,1,1
1,1,True,True,December,31,2


In [52]:
len(temp_df)==len(temp_df[((temp_df['date_type_id'] == 1)
         & (temp_df['month_name_desc'] == 'January')
         & (temp_df['date_ref_day_number'] == 1)) &
        (temp_df['date_ref_fixed'] == 1) |
        ((temp_df['date_type_id'] == 2)
         & (temp_df['month_name_desc'] == 'December')
        & (temp_df['date_ref_day_number'] == 31)) &
        (temp_df['date_ref_fixed'] == 1)])

True

In [53]:
results.append(
    len(temp_df) == len(temp_df[((temp_df['date_type_id'] == 1)
                                 & (temp_df['month_name_desc'] == 'January')
                                 & (temp_df['date_ref_day_number'] == 1))
                                & (temp_df['date_ref_fixed'] == 1)
                                | ((temp_df['date_type_id'] == 2)
                                   & (temp_df['month_name_desc'] == 'December')
                                   & (temp_df['date_ref_day_number'] == 31))
                                & (temp_df['date_ref_fixed'] == 1)]))

### Seventh before test

In [54]:
timing.append('before')

In [55]:
# before_season_definition

In [56]:
descriptions.append('''For periodic seasons, test that the difference between \
start/end months is between 2 and 10 inclusive''')

In [57]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'date_ref_fixed', 'month_name_desc',
        'date_ref_day_number', 'date_type_id'
    ]]
temp_df

,season_id,year_round,date_ref_fixed,month_name_desc,date_ref_day_number,date_type_id
0,2,False,True,May,1,1
1,2,False,True,October,1,2
2,3,False,True,March,1,1
3,3,False,True,October,31,2


In [58]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id',  'month_name_desc',
        'date_type_id'
    ]].merge(month_tbl[['month_name_desc','month_name_num']], on='month_name_desc')
temp_df

,season_id,month_name_desc,date_type_id,month_name_num
0,2,May,1,5
1,2,October,2,10
2,3,October,2,10
3,3,March,1,3


In [59]:
temp_df2 = temp_df.sort_values(by='date_type_id').groupby('season_id').first().merge(
    temp_df.sort_values(by='date_type_id').groupby('season_id').last(),
    left_index=True,
    right_index=True)[['month_name_num_x',
                       'month_name_num_y']].rename(columns={
                           'month_name_num_x': 'start',
                           'month_name_num_y': 'end'
                       })
temp_df2

,start,end
season_id,,
2,5,10
3,3,10


In [60]:
temp_df2 = temp_df.sort_values(by='date_type_id').groupby('season_id').first().merge(
    temp_df.sort_values(by='date_type_id').groupby('season_id').last(),
    left_index=True,
    right_index=True)[['month_name_num_x',
                       'month_name_num_y']].rename(columns={
                           'month_name_num_x': 'start',
                           'month_name_num_y': 'end'
                       })

In [61]:
def get_diff(row, col_strt, col_end):
    if row[col_end]-row[col_strt]>0:
        return row[col_end]-row[col_strt]
    elif row[col_end]-row[col_strt]<0:
        return 12 + row[col_end]-row[col_strt]

In [62]:
temp_df2['diff'] = temp_df2.apply(lambda row: get_diff(row, 'start', 'end'), axis = 1)

In [63]:
range_vals = []
for i in temp_df2['diff'].values:
    if (i < 2) | (i > 10):
        range_vals.append(False)
    else:
        range_vals.append(True)           

In [64]:
# range_vals

In [65]:
results.append(False not in range_vals)

### Eighth before test

In [66]:
timing.append('before')

In [67]:
descriptions.append('''For year-round seasons, test that 'effective_start' \
in tbl_sla_season_date is equal to 'effective_start' in tbl_sla_season''')

In [68]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == True],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_start_y'
    ]]
temp_df

,season_id,year_round,effective_start_x,effective_start_y
0,1,True,2014-01-01,2014-01-01


In [69]:
results.append(
    len(temp_df) == len(temp_df[temp_df['effective_start_x'] ==
                                temp_df['effective_start_y']]))

### Ninth before test

In [70]:
timing.append('before')

In [71]:
descriptions.append('''For periodic seasons, test that the earliest \
'effective_start' for each season in tbl_sla_season_date is equal to \
'effective_start' in tbl_sla_season''')

In [72]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_start_y'
    ]].sort_values(by='effective_start_x').groupby('season_id').first()
temp_df

,year_round,effective_start_x,effective_start_y
season_id,,,
2,False,2019-07-01,2019-07-01
3,False,2019-07-01,2019-07-01


In [73]:
results.append(
    len(temp_df) == len(temp_df[temp_df['effective_start_x'] ==
                                temp_df['effective_start_y']]))

### Tenth before test

In [74]:
timing.append('before')

In [75]:
descriptions.append('''For periodic seasons, test that the latest \
'effective_end' for each season in tbl_sla_season_date is equal to \
'effective_end' in tbl_sla_season OR today+1year (if null)''')

In [76]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_end_x', 'effective_end_y'
    ]].sort_values(by='effective_start_x').groupby('season_id').last()
temp_df

,year_round,effective_start_x,effective_end_x,effective_end_y
season_id,,,,
2,False,2021-05-01,2021-09-10,NaT
3,False,2021-03-01,2021-09-10,NaT


In [77]:
two_conditions = []

In [78]:
two_conditions.append(len(temp_df[~pd.isnull(temp_df['effective_end_y'])]) == len(
    temp_df[(~pd.isnull(temp_df['effective_end_y']))
            & (temp_df['effective_end_x'] == temp_df['effective_end_y'])]))

In [79]:
two_conditions.append(
    len(temp_df[pd.isnull(temp_df['effective_end_y'])]) == len(temp_df[
        (pd.isnull(temp_df['effective_end_y']))
        & (temp_df['effective_end_x'] == datetime.datetime(
            today.year + 1, today.month, today.day))]))

In [80]:
results.append(False not in two_conditions)

### Eleventh before test

In [81]:
timing.append('before')

In [82]:
descriptions.append('''Test to make sure there are no gaps in dates \
in tbl_sla_season_date''')

In [83]:
sql_gaps_1 = '''
select season_id,
       datediff(day, min(effective_start), max(effective_end)) as daydiff
from sladb.dbo.tbl_sla_season_date
group by season_id
'''

sql_gaps_2 = '''select season_id,
       case when count(*) > 1 then sum(daydiff) + (count(*) - 1)
            else sum(daydiff)
       end as daydiff
from (select season_id,
             datediff(day, effective_start, effective_end) as daydiff
      from sladb.dbo.tbl_sla_season_date) as t
group by season_id
'''

In [84]:
gaps_df1 = pd.read_sql(con = engine, sql = sql_gaps_1)
gaps_df2 = pd.read_sql(con = engine, sql = sql_gaps_2)

In [85]:
temp_df = gaps_df1.merge(gaps_df2, on='season_id')

In [86]:
results.append(len(temp_df)==len(temp_df[temp_df['daydiff_x']==temp_df['daydiff_y']]))

### Twelfth before test

In [87]:
timing.append('before')

In [88]:
descriptions.append('''Test to make sure there are no gaps in dates \
('effective_start_adj' and 'effective_end_adj')in tbl_sla_season_date''')

In [89]:
sql_gaps_1 = '''
select season_id,
       datediff(day, min(effective_start_adj), max(effective_end_adj)) as daydiff
from sladb.dbo.tbl_sla_season_date
group by season_id
'''

sql_gaps_2 = '''select season_id,
       case when count(*) > 1 then sum(daydiff) + (count(*) - 1)
            else sum(daydiff)
       end as daydiff
from (select season_id,
             datediff(day, effective_start_adj, effective_end_adj) as daydiff
      from sladb.dbo.tbl_sla_season_date) as t
group by season_id
'''

In [90]:
gaps_df1 = pd.read_sql(con = engine, sql = sql_gaps_1)
gaps_df2 = pd.read_sql(con = engine, sql = sql_gaps_2)

In [91]:
temp_df = gaps_df1.merge(gaps_df2, on='season_id')

In [92]:
temp_df[temp_df['daydiff_x']!=temp_df['daydiff_y']]

,season_id,daydiff_x,daydiff_y


In [93]:
len(temp_df)==len(temp_df[temp_df['daydiff_x']==temp_df['daydiff_y']])

True

In [94]:
results.append(len(temp_df)==len(temp_df[temp_df['daydiff_x']==temp_df['daydiff_y']]))

### Twelfth before test

In [95]:
timing.append('before')

In [96]:
descriptions.append('''In tbl_unit_sla_season, test to make sure \
the 'effective_start_adj' is between 'effective_start_adj' and \
'effective_end_adj' from tbl_sla_season if not null. If 'effective_end_adj' \
is null, replace with today's date+1year''')

In [97]:
temp_df = before_unit_season.merge(before_sla_season[[
    'season_id', 'effective_start_adj', 'effective_end_adj'
]],
                                   on='season_id')[[
                                       'unit_id', 'season_id',
                                       'effective_start_adj_x',
                                       'effective_start_adj_y',
                                       'effective_end_adj_y'
                                   ]].copy()
temp_df.head()

,unit_id,season_id,effective_start_adj_x,effective_start_adj_y,effective_end_adj_y
0,RZ245,1,2014-01-05,2014-01-05,NaT
1,RZ25,1,2014-01-05,2014-01-05,NaT
2,RZ255,1,2014-01-05,2014-01-05,NaT
3,RZ258,1,2014-01-05,2014-01-05,NaT
4,RZ26,1,2014-01-05,2014-01-05,NaT


In [98]:
temp_df['effective_end_adj_y'].fillna(datetime.datetime(
            today.year + 1, today.month, today.day), inplace=True)

In [99]:
results.append(
    len(temp_df[
        (temp_df['effective_start_adj_x'] < temp_df['effective_start_adj_y'])
        | (temp_df['effective_start_adj_x'] > temp_df['effective_end_adj_y'])])
    == 0)

### Thirteenth before test

In [100]:
timing.append('before')

In [101]:
descriptions.append('''In tbl_unit_sla_season, test to make sure the \
'effective_end_adj' is between 'effective_start_adj' and 'effective_end_adj' \
from tbl_sla_season if not null. If 'effective_end_adj' is null, replace with \
today's date+1year''')

In [102]:
temp_df = before_unit_season.merge(before_sla_season[[
    'season_id', 'effective_start_adj', 'effective_end_adj'
]],
                                   on='season_id')[[
                                       'unit_id', 'season_id',
                                       'effective_end_adj_x',
                                       'effective_start_adj_y',
                                       'effective_end_adj_y'
                                   ]].copy()
temp_df.head()

,unit_id,season_id,effective_end_adj_x,effective_start_adj_y,effective_end_adj_y
0,RZ245,1,2020-03-21,2014-01-05,NaT
1,RZ25,1,2020-03-21,2014-01-05,NaT
2,RZ255,1,2020-03-21,2014-01-05,NaT
3,RZ258,1,2020-03-21,2014-01-05,NaT
4,RZ26,1,2020-03-21,2014-01-05,NaT


In [103]:
temp_df['effective_end_adj_y'].fillna(datetime.datetime(
            today.year + 1, today.month, today.day), inplace=True)

In [104]:
temp_df[
        (temp_df['effective_end_adj_x'] < temp_df['effective_start_adj_y'])
        | (temp_df['effective_end_adj_x'] > temp_df['effective_end_adj_y'])]

,unit_id,season_id,effective_end_adj_x,effective_start_adj_y,effective_end_adj_y


In [105]:
temp_df[
        (temp_df['effective_end_adj_x'] < temp_df['effective_start_adj_y'])
        | (temp_df['effective_end_adj_x'] > temp_df['effective_end_adj_y'])]['season_id'].unique()

array([], dtype=int64)

In [106]:
results.append(
    len(temp_df[
        (temp_df['effective_end_adj_x'] < temp_df['effective_start_adj_y'])
        | (temp_df['effective_end_adj_x'] > temp_df['effective_end_adj_y'])])
    == 0)

## Make Updates

## Execute all of the sql queries AFTER UPDATES and store the results

## List of tuples with final test results

In [107]:
list(zip(timing, descriptions, results))

[('before',
  "Ensure that 'effective'==1 for every season that has 'effective_end_date' == null",
  True),
 ('before',
  "Test that 'effective_end_adj' is after today if 'effective'==1 AND 'effective_end' != null in tbl_sla_season",
  True),
 ('before',
  'Test that every season in tbl_sla_season has exactly two entries in tbl_ref_sla_season_definition',
  True),
 ('before',
  "Test that in tbl_ref_sla_season_definition if 'date_ref_fixed'==True then columns 'month_name_desc' and 'date_ref_day_number' are not null",
  True),
 ('before',
  "Test that in tbl_ref_sla_season_definition if 'date_ref_fixed'==False then columns 'month_name_desc' and 'day_name_desc' and 'day_rank_id' are not null",
  True),
 ('before',
  "For year-round seasons, test that 'date_ref_fixed'==1, and start/end dates are Jan 1/Dec 31",
  True),
 ('before',
  'For periodic seasons, test that the difference between start/end months is between 2 and 10 inclusive',
  True),
 ('before',
  "For year-round seasons, test 

In [108]:
pd.DataFrame(list(zip(timing, descriptions, results)), columns = {'before/after', 'test_desc', 'result'})             

,before/after,result,test_desc
0,before,Ensure that 'effective'==1 for every season th...,True
1,before,Test that 'effective_end_adj' is after today i...,True
2,before,Test that every season in tbl_sla_season has e...,True
3,before,Test that in tbl_ref_sla_season_definition if ...,True
4,before,Test that in tbl_ref_sla_season_definition if ...,True
5,before,"For year-round seasons, test that 'date_ref_fi...",True
6,before,"For periodic seasons, test that the difference...",True
7,before,"For year-round seasons, test that 'effective_s...",True
8,before,"For periodic seasons, test that the earliest '...",True
9,before,"For periodic seasons, test that the latest 'ef...",True
